In [7]:
!pip install selenium langchain-groq

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
import re
import json
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import requests
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
load_dotenv()

chat = ChatGroq(temperature=0, groq_api_key=os.getenv("GROQ_API_KEY"), model_name="llama3-8b-8192")
parser = StrOutputParser()


# Initialize Chrome options
chrome_options = Options()
# Set up date formatting for today's date
today = datetime.today().strftime('%Y-%m-%d')


In [9]:
linkedin_credentials = json.load(open("/home/jellyfish/Large Language Models (LLM)/Scrapper/linkedin_credentials.json"))
username = linkedin_credentials['username']
password = linkedin_credentials['password']

In [10]:
browser = webdriver.Chrome()

#Open login page
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
browser.maximize_window()
wait = WebDriverWait(browser, 20)

In [11]:
#Enter login info:
elementID = browser.find_element(By.ID, "username")
elementID.send_keys(username)

elementID = browser.find_element(By.ID, "password")
elementID.send_keys(password)

elementID.submit()

In [12]:
WebDriverWait(browser,4).until(EC.presence_of_all_elements_located((By.XPATH,'//*[@id="global-nav-typeahead"]/input')))
browser.find_element(By.XPATH,'//*[@id="global-nav-typeahead"]/input').send_keys('Looking for offshore software Development company') #####################@@@@@@@@@@@@@@@@@

TimeoutException: Message: 


In [ ]:
# browser.find_element(By.XPATH,'//*[@id="global-nav-typeahead"]/input').send_keys(Keys.ENTER)

element = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="global-nav-typeahead"]/input')))
element.send_keys(Keys.ENTER)

In [ ]:
# Wait until the element is clickable
element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search-reusables__filters-bar"]/ul/li[1]/button')))

# Click on the element
element.click()

In [ ]:
element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchFilter_datePosted"]')))

# Click on the element
element.click()

In [ ]:
element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="datePosted-past-week"]')))

# Execute script to click on the element
browser.execute_script("arguments[0].click();", element)

In [ ]:
element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div[3]/div[2]/section/div/nav/div/ul/li[4]/div/div/div/div[1]/div/form/fieldset/div[2]/button[2]')))

# Click on the element
element.click()

In [ ]:
def validate(post:str):
    system = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

    Given a text excerpt from a social media post or online article, determine whether it indicates a potential business opportunity or interest in Jellyfish Technologies' software outsourcing services. Relevant posts should suggest a need for software development, AI solutions, or data analytics services, aligning with Jellyfish Technologies' expertise.

Examples of relevant indicators include:

    Businesses expressing a need for custom software solutions or AI development.
    Organizations discussing challenges in data management or analysis that Jellyfish Technologies can address.
    Posts seeking recommendations for reliable software outsourcing partners.

Examples of irrelevant indicators include:

    Personal anecdotes unrelated to business or technology.
    General news updates without relevance to software development, AI, or data analytics.
    Posts promoting products or services outside of Jellyfish Technologies' scope.
    ****Do not Assume anything by yourself. Answer based on post only***
    

    **Output Format***
    Yes: summary of post
    No : No
    <|eot_id|>
    """
    human = """<|start_header_id|>Client Post<|end_header_id|>

    {post}

    Ouput summary only if it is useful for getting sales <|eot_id|> "
    """

    prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
    chain = prompt | chat | parser
    output = chain.invoke({"post":post})
    return output

In [ ]:
import time

SCROLL_PAUSE_TIME = 5
MAX_SCROLLS = 50

# JavaScript commands
SCROLL_COMMAND = "window.scrollTo(0, document.body.scrollHeight);"
GET_SCROLL_HEIGHT_COMMAND = "return document.body.scrollHeight"

# Initial scroll height
last_height = browser.execute_script(GET_SCROLL_HEIGHT_COMMAND)
scrolls = 0
no_change_count = 0

while True:
    try:
        browser.find_element(By.Id, '//*[@id="ember1044"]/div/div[4]/div/button').click()
    except:
        pass
    # Scroll down to bottom
    browser.execute_script(SCROLL_COMMAND)

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script(GET_SCROLL_HEIGHT_COMMAND)
    
    # Increment no change count or reset it
    no_change_count = no_change_count + 1 if new_height == last_height else 0
    
    # Break loop if the scroll height has not changed for 3 cycles or reached the maximum scrolls
    if no_change_count >= 3 or (MAX_SCROLLS and scrolls >= MAX_SCROLLS):
        break
    
    last_height = new_height
    scrolls += 1

In [ ]:
company_page = browser.page_source
linkedin_soup = bs(company_page.encode("utf-8"), "html")

with open(f"soup.txt", "w+") as t:
    t.write(linkedin_soup.prettify())
#print(linkedin_soup.prettify())

containers = linkedin_soup.find_all("div",{"class":"feed-shared-update-v2"})
containers = [container for container in containers if 'activity' in container.get('data-urn', '')]
print(len(containers))

containers_text = "\n\n".join([c.prettify() for c in containers])
with open(f"soup_containers.txt", "w+") as t:
    t.write(containers_text)

188


In [ ]:
def extract_numerical_value(date_str):
    # Use regular expression to find the numerical value
    numerical_value = re.findall(r'\d+', date_str)
    # Convert the first numerical value found to an integer
    if numerical_value:
        return int(numerical_value[0])
    else:
        return None
    
def get_actual_date(date):
    today = datetime.today().strftime('%Y-%m-%d')
    current_year = datetime.today().strftime('%Y')
    
    def get_past_date(days=0, weeks=0, months=0, years=0):
        date_format = '%Y-%m-%d'
        dtObj = datetime.strptime(today, date_format)
        past_date = dtObj - relativedelta(days=days, weeks=weeks, months=months, years=years)
        past_date_str = past_date.strftime(date_format)
        return past_date_str

    past_date = date
    
    if 'hour' in date:
        past_date = today
    elif 'day' in date:
        past_date = get_past_date(days=extract_numerical_value(date))
    elif 'week' in date:
        past_date = get_past_date(weeks=extract_numerical_value(date))
    elif 'month' in date:
        past_date = get_past_date(months=extract_numerical_value(date))
    elif 'year' in date:
        past_date = get_past_date(months=extract_numerical_value(date))
    else:
        split_date = date.split("-")
        if len(split_date) == 2:
            past_month = split_date[0]
            past_day =  split_date[1]
            if len(past_month) < 2:
                past_month = "0"+past_month
            if len(past_day) < 2:
                past_day = "0"+past_day
            past_date = f"{current_year}-{past_month}-{past_day}"
        elif len(split_date) == 3:
            past_month = split_date[0]
            past_day =  split_date[1]
            past_year = split_date[2]
            if len(past_month) < 2:
                past_month = "0"+past_month
            if len(past_day) < 2:
                past_day = "0"+past_day
            past_date = f"{past_year}-{past_month}-{past_day}"

    return past_date



def convert_abbreviated_to_number(s):
    if 'K' in s:
        return int(float(s.replace('K', '')) * 1000)
    elif 'M' in s:
        return int(float(s.replace('M', '')) * 1000000)
    else:
        return int(s)

In [ ]:
import re

# Define a data structure to hold all the post information
posts_data = []

def get_text(container, selector, attributes):
    try:
        element = container.find(selector, attributes)
        if element:
            text = element.text.strip()
            # Check if there's a "See more" link
            see_more_link = element.find("span", {"class": "see-more"})
            if see_more_link:
                # Click on the "See more" link to expand the text
                see_more_link.click()
                # Get the expanded text
                expanded_text = element.text.strip()
                # Replace the original text with the expanded text
                text = expanded_text
            return text
    except Exception as e:
        print(e)
    return ""

# Assuming containers is a list of BeautifulSoup elements representing posts

for container in containers:
    post_text = get_text(container, "span", {"class": "break-words"})
    post_date = get_text(container, "span", {"class": "update-components-actor__sub-description"})
    # post_date = extract_date_time(post_date)
    post_date = get_actual_date(post_date)
    print(post_date)
    
    
       # Extract profile name and profile link
    profile_name_element = container.find("span", {"class": "text-view-model"})
    profile_name = profile_name_element.text.strip() if profile_name_element else "Unknown"
    
    profile_link_element = container.find("a", {"class": "app-aware-link"})
    profile_link = profile_link_element['href'] if profile_link_element else "Unknown"
    
    # Check if email is present in the post text
    email = re.search(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', post_text)
    email = email.group() if email else None
    sumarize = validate(post_text)
    if ':' in sumarize:
        sumarize = sumarize.split(':')[1]
    time.sleep(1)
    try:
        posts_data.append({
                "Date": post_date,
                "Profile Name": profile_name,
                "Profile Link": profile_link,
                "Post Text": post_text,
                "post summary": sumarize,
                "Email": email
            })
    except:
        pass

2024-06-18
2023-10-02
2024-06-02
2024-06-02
2024-06-02
2023-09-02
2024-02-02


In [ ]:
posts_data

[{'Date': '2024-05-16',
  'Profile Name': 'Techysquad',
  'Profile Link': 'https://www.linkedin.com/company/techysquad-technology/?miniCompanyUrn=urn%3Ali%3Afs_miniCompany%3A26585317',
  'Post Text': 'Looking for a convenient way to cut down the time-to-market for your new web app? Outsource PHP Web Development. When budgets are tight, opting for an experienced team in a nearshore or offshore location can help optimise your IT costs.Selecting Techysquad as your outsourcing destination provides access to skilled IT engineers, top-notch software development, and chances to build fruitful partnerships.Read the article to learn more about this.#Techysquad #PHP #PhpWebDevelopment #Webdesign #webdevelopment #phpsite #digitalmarketing #Digitalagency',
  'post summary': 'No',
  'Email': None},
 {'Date': '2024-05-14',
  'Profile Name': 'Get Devs',
  'Profile Link': 'https://www.linkedin.com/company/getdevs/?miniCompanyUrn=urn%3Ali%3Afs_miniCompany%3A3281577',
  'Post Text': 'Struggling with a t

In [ ]:
try:
    df = pd.DataFrame(posts_data)
except:
    for item in list(posts_data.keys()):
        print(item)
        print(len(posts_data[item]))
        
for col in df.columns:
    try:
        df[col] = df[col].astype(int)
    except:
        pass
    
df.sort_values(by="Date", inplace=True, ascending=False)
df

,Date,Profile Name,Profile Link,Post Text,post summary,Email
0,2024-05-16,Techysquad,https://www.linkedin.com/company/techysquad-te...,Looking for a convenient way to cut down the t...,No,None
12,2024-05-16,Dimitra Karamesini,https://www.linkedin.com/in/dimitrakaramesini?...,Shipping Telegraph News: New Edition Published...,No,None
24,2024-05-16,The Business Street,https://www.linkedin.com/company/the-business-...,#marketresearch #businessinsights #dataanalysi...,No,None
20,2024-05-16,InfoQuest Research Solutions,https://www.linkedin.com/company/infoquest-res...,#MarketInsights #TechnologyTrends #IndustryAna...,No,None
19,2024-05-16,Xuelai ZHANG,https://www.linkedin.com/in/bottom-up-xl?miniP...,"In 2023, China demonstrated a robust balance o...",No,None
15,2024-05-16,Swappsi software,https://www.linkedin.com/company/swappsi-softw...,The wonderful adventure of creating your own s...,No,None
13,2024-05-16,Michael Olsen,https://www.linkedin.com/in/miolsen?miniProfil...,Shipping Telegraph News: New Edition Published...,No,None
28,2024-05-16,Tech News And Updates,https://www.linkedin.com/company/tech-news-and...,,No,None
4,2024-05-16,Naresh Kolimela,https://www.linkedin.com/in/naresh-kolimela-99...,Position : Sr. Golang DeveloperLocation: Cinci...,The post is about a job opening for a Sr. Gol...,naresh@equitasitinc.com
5,2024-05-16,Zafrulla Rasheed,https://www.linkedin.com/in/zafrulla-rasheed-4...,"In this thought provoking piece, we explore th...",The post discusses the integration of advance...,None


In [ ]:
df.to_csv(f"{today}_posts.csv", encoding='utf-8', index=False)


In [ ]:
df

,Date,Profile Name,Profile Link,Post Text,post summary,Email
0,2024-05-16,Techysquad,https://www.linkedin.com/company/techysquad-te...,Looking for a convenient way to cut down the t...,No,None
12,2024-05-16,Dimitra Karamesini,https://www.linkedin.com/in/dimitrakaramesini?...,Shipping Telegraph News: New Edition Published...,No,None
24,2024-05-16,The Business Street,https://www.linkedin.com/company/the-business-...,#marketresearch #businessinsights #dataanalysi...,No,None
20,2024-05-16,InfoQuest Research Solutions,https://www.linkedin.com/company/infoquest-res...,#MarketInsights #TechnologyTrends #IndustryAna...,No,None
19,2024-05-16,Xuelai ZHANG,https://www.linkedin.com/in/bottom-up-xl?miniP...,"In 2023, China demonstrated a robust balance o...",No,None
15,2024-05-16,Swappsi software,https://www.linkedin.com/company/swappsi-softw...,The wonderful adventure of creating your own s...,No,None
13,2024-05-16,Michael Olsen,https://www.linkedin.com/in/miolsen?miniProfil...,Shipping Telegraph News: New Edition Published...,No,None
28,2024-05-16,Tech News And Updates,https://www.linkedin.com/company/tech-news-and...,,No,None
4,2024-05-16,Naresh Kolimela,https://www.linkedin.com/in/naresh-kolimela-99...,Position : Sr. Golang DeveloperLocation: Cinci...,The post is about a job opening for a Sr. Gol...,naresh@equitasitinc.com
5,2024-05-16,Zafrulla Rasheed,https://www.linkedin.com/in/zafrulla-rasheed-4...,"In this thought provoking piece, we explore th...",The post discusses the integration of advance...,None
